# Inference Pipeline for trained models

## Utils

In [ ]:
print("optimal image size: 4736 1920")
w = 1920
h = 4736
sizes = [(h, w), (h//2, w//2), (h//4, w//4), (h//8, w//8)]
print(sizes[0], sizes[1], sizes[2], sizes[3])
size = sizes[2]

optimal image size: 4736 1920
(4736, 1920) (2368, 960) (1184, 480) (592, 240)


In [ ]:
import os
import pydicom
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class DatasetNew(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return {"image": self.data[index]['image'], "mask": self.data[index]['mask']}

In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from functionality import *

path_to_dataset = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\Ver1_ 4736x1920\\NIITO_Vertebrae_Dataset_Test_resized"
#  "C:\Users\EUgolnikova\Dropbox\NIITO_Vertebrae\NIITO_Vertebrae_Dataset\NIITO_Vertebrae_Dataset_Test"

path_to_images = os.path.join(path_to_dataset, "images")
path_to_labels = os.path.join(path_to_dataset, "labels")

test_cases = os.listdir(path_to_images)

test_transforms = A.ReplayCompose(
    [   
        A.Resize(height=size[0], width=size[1]),
        ToTensorV2()
    ],
    additional_targets={'image': 'image', 'mask': 'mask'})



test_aug = []
for case in test_cases:
    print(case)
    path_mask = os.path.join(path_to_labels, *[case, "fill_no_FH", case + "_SD.png"])
    path_image = os.path.join(path_to_images, *[case, case + "_SD_resized.dcm"])
    
    image, mask, _ = read_image(path_image, path_mask, channels3=True)
    mask[mask==255.0] = 1.0
    # print(image.shape, mask.shape)

    # image = np.moveaxis(image, 0, 2)

    
    augmentations = test_transforms(image=image, mask=mask)
    # print("!!",augmentations['image'].shape, augmentations['mask'].shape)

    test_aug.append({
        "image": augmentations['image'],
        "mask": augmentations['mask']
    })


test_dataset = DatasetNew(test_aug)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, pin_memory=True, shuffle=True)

print(len(test_dataset))

053
057
060
063
073
5


In [ ]:
def get_mean_std(loader):
    ch_sum, ch_squared_sum, count_of_batches = 0, 0, 0
    
    for data in loader:
        data = data['image'].float()
        data /= 255        

        ch_sum += torch.mean(data, dim=[0, 2, 3])
        ch_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        count_of_batches += 1

    mean = ch_sum / count_of_batches 
    std = (ch_squared_sum / count_of_batches - mean**2)**0.5

    return mean, std 


def soft_dice(*, y_true, y_pred):
    eps = 1e-15
    y_pred = y_pred.contiguous().view(y_pred.numel())
    y_true = y_true.contiguous().view(y_true.numel())
    intersection = (y_pred * y_true).sum(0)
    scores = 2. * (intersection + eps) / (y_pred.sum(0) + y_true.sum(0) + eps)
    score = scores.sum() / scores.numel()
    
    return torch.clamp(score, 0., 1.)


def hard_dice(*, y_true, y_pred, thr=0.5):
    y_pred = (y_pred > thr).float()
    return soft_dice(y_true=y_true, y_pred=y_pred)


def accuracy(y_true, y_pred, thr=0.5):
    num_correct = 0
    num_pixels = 0
    
    y_pred = (y_pred > thr).float()
    num_correct += (y_true == y_pred).sum()
    num_pixels += torch.numel(y_pred)
    
    return num_correct/num_pixels*100



In [ ]:
import torch
import torchvision
import skimage
import numpy as np
import time


def read_mask(mask_name):
    mask = (skimage.io.imread(mask_name)[:,:]==255).astype(np.uint8)*255
    mask = (mask > 0).astype(np.uint8)
    return mask



def make_blending(img_path, mask_path, alpha=0.5):
    img, mask = read_mask(img_path), read_mask(mask_path)[:, :, 0]
    colors = np.array([[0,0,0], [255,0,0]], np.uint8)
    return (img*alpha + colors[mask.astype(np.int32)]*(1. - alpha)).astype(np.uint8)


def show_images_with_mask(image_path,  mask_path_fill, alpha=0.5):
    plt.figure(figsize=(20, 14))
    plt.subplot(1, 2, 1)
    orig, _m = read_mask(image_path), read_mask(mask_path_fill)
    plt.imshow(orig)
    plt.subplot(1, 2, 2)
    blend = make_blending(image_path, mask_path_fill, alpha)
    plt.imshow(blend)


def save_predictions_as_imgs(loader, model, thr=0.5, folder="/content/saved_images", device='cpu'):
  if not os.path.exists(folder):
    os.makedirs(folder)

  model.to(device=device).eval()
#   model.eval()
  acc = []
  s_dice = []
  h_dice = []
  times = []
  for idx, data in enumerate(loader):

    x = img = data['image'].float().to(device=device)
    img = torch.squeeze(img, 0)
    img = img.permute(1, 2, 0)
    y = mask = data['mask'].to(device=device)
    mask = torch.squeeze(mask, 0)

    with torch.no_grad():
      start_time= time.time() 
      preds = torch.sigmoid(model(x))
      preds = (preds > thr).float()
      stop_time=time.time()

    duration =stop_time - start_time
    hours = duration // 3600
    minutes = (duration - (hours * 3600)) // 60
    seconds = duration - ((hours * 3600) + (minutes * 60))
    msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
    times.append(duration)

    x = x.float() / 255
    # print(type(x), type(preds))
    acc.append(accuracy(y, preds))
    h_dice.append(hard_dice(y_true=y, y_pred=preds))
    s_dice.append(soft_dice(y_true=y, y_pred=preds))

    torchvision.utils.save_image(x, f"{folder}/orig_{idx}.png")
    torchvision.utils.save_image(preds, f"{folder}/pred_{idx}.png")
    torchvision.utils.save_image(y.float(), f"{folder}/gt_{idx}.png")

    # torchvision.utils.save_image(y, f"{folder}/{idx}.png")
  
  means = [np.mean(acc), np.mean(s_dice), np.mean(h_dice), np.mean(times)]
  return {"accuracy": acc, "soft DICE": s_dice, "DICE": h_dice, "time": times, "means": means} 
  


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt

def save_blended(path_to_folder, number_of_images, acc = None, s_dice = None, h_dice = None, alpha = 0.75, beta = 0.95):

    for k in range(number_of_images):
        path_img = os.path.join(path_to_folder, "orig_" + str(k) + ".png")
        path_mask = os.path.join(path_to_folder, "pred_" + str(k) + ".png")
        path_mask_gt = os.path.join(path_to_folder, "gt_" + str(k) + ".png")
        path_save = os.path.join(path_to_folder, "blend_" + str(k) + ".png")

        img = plt.imread(path_img)
        msk = plt.imread(path_mask)
        msk_gt = plt.imread(path_mask_gt)

        red  = np.array([1,0,0],dtype=np.uint8)
        blue = np.array([0,0,1],dtype=np.uint8)

        # print(accuracy(img, msk))
        # print(hard_dice(y_true=img, y_pred=msk))
        # print(soft_dice(y_true=img, y_pred=msk))

        # plt.figure(figsize=(20, 14))
        # plt.subplot(1, 3, 1)
        # plt.imshow(img)
        # plt.subplot(1, 3, 2)
        # plt.imshow(msk)

        # print(img.shape, msk.shape)
        for i in range(msk.shape[0]):
            for j in range(msk.shape[1]):
                if msk[i, j].all() == 1:
                    msk[i, j] = red
                if msk_gt[i, j].all() == 1:
                    msk_gt[i, j] = blue

        # for i in range(msk.shape[0]):
        #     for j in range(msk.shape[1]):
        #         if msk[i, j].all() == 1:
        #             msk[i, j] = red
                

        res = (img*alpha + msk*(1 - alpha))
        res = (res*beta + msk_gt*(1 - beta))


        # plt.subplot(1, 3, 3)
        # plt.imshow(res)


        res = (res * 255).astype(np.uint8)

        im = Image.fromarray(res)

        str_ = ""
        str_ += f"\n accuracy: {acc[k]}" if acc is not None else ""
        str_ += f"\n DICE: {h_dice[k]}" if h_dice is not None else ""

        # print(str_)
        font = ImageFont.truetype('arial', size=28)
        ImageDraw.Draw(
            im  # Image
        ).text(
            (0, 0),  # Coordinates
            str_,  # Text
            (255, 185, 93),  # Color
            font
        )


        im.save(path_save)


## Model

In [ ]:
import torch
import torch.nn as nn


class ConvLRelu(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False)
        self.batchNorm = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchNorm(x)
        x = self.activation(x)
        return x

    
class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            ConvLRelu(in_channels, out_channels),
            ConvLRelu(out_channels, out_channels),
        )
    
    def forward(self, x):
        x = self.conv_block(x)
        return x

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv_block = DoubleConvBlock(in_channels, out_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
            
    def forward(self, x):
        before_pool = self.conv_block(x)
        x = self.max_pool(before_pool)
        return x, before_pool
    
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()              
        self.conv_block = DoubleConvBlock(in_channels, out_channels)

    def forward(self, x, y):
        x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return self.conv_block(torch.cat([x, y], dim=1))

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, n_filters=64):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)

        self.enc1 = EncoderBlock(in_channels, n_filters)
        self.enc2 = EncoderBlock(n_filters, n_filters * 2)
        self.enc3 = EncoderBlock(n_filters * 2, n_filters * 4)
        self.enc4 = EncoderBlock(n_filters * 4, n_filters * 8)
        
        self.center = DoubleConvBlock(n_filters * 8, n_filters * 16)
        
        self.dec4 = DecoderBlock(n_filters * (16 + 8), n_filters * 8)
        self.dec3 = DecoderBlock(n_filters * (8 + 4), n_filters * 4)
        self.dec2 = DecoderBlock(n_filters * (4 + 2), n_filters * 2)
        self.dec1 = DecoderBlock(n_filters * (2 + 1), n_filters)

        self.final = nn.Conv2d(n_filters, out_channels, kernel_size=1)

    def forward(self, x):
        x = x.float()
        x, enc1 = self.enc1(x)
        x, enc2 = self.enc2(x)
        x, enc3 = self.enc3(x)
        x, enc4 = self.enc4(x)

        center = self.center(x)

        dec4 = self.dec4(center, enc4)
        dec3 = self.dec3(dec4, enc3)
        dec2 = self.dec2(dec3, enc2)
        dec1 = self.dec1(dec2, enc1)

        
        final = self.final(dec1)

        return final

# Inference 

## Morning Fire 16



In [ ]:
hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 1e-3,
    'epochs': 50,
    'lr_reduce_rate': 0.5,
    'patience': 4,
    'early_stopping': 50, # пока уберем раннюю остановку
    'model': 'test'
}
model = UNet(n_filters=hyperparametrs['n_filters'])
model.load_state_dict(torch.load('C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/weight/UNet_spinal-cord/morning-fire-16/weights.pth', map_location=torch.device('cpu')))

folder = 'C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/inference/UNet_spinal-cord/morning-fire-16/'
metrics = save_predictions_as_imgs(test_loader, model, folder = folder)

KeyboardInterrupt: 

## Classic Pine 17

## Swift Sunset 18

In [ ]:
hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 1e-3,
    'epochs': 50,
    'lr_reduce_rate': 0.5,
    'patience': 4,
    'early_stopping': 50, # пока уберем раннюю остановку
    'model': 'test'
}
model = UNet(n_filters=hyperparametrs['n_filters'])
model.load_state_dict(torch.load('C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/weight/UNet_spinal-cord/swift-sunset-18/weights.pth', map_location=torch.device('cpu')))

folder = 'C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/inference/UNet_spinal-cord/swift-sunset-18/'
metrics = save_predictions_as_imgs(test_loader, model, folder = folder)

save_blended(path_to_folder=folder, number_of_images=5, acc = metrics['accuracy'], h_dice = metrics['DICE'], alpha = 0.75, beta = 0.8)


In [ ]:
metrics

{'accuracy': [tensor(98.6852),
  tensor(98.8316),
  tensor(99.2490),
  tensor(99.2589),
  tensor(99.1139)],
 'soft DICE': [tensor(0.8779),
  tensor(0.8455),
  tensor(0.8958),
  tensor(0.9120),
  tensor(0.8862)],
 'DICE': [tensor(0.8779),
  tensor(0.8455),
  tensor(0.8958),
  tensor(0.9120),
  tensor(0.8862)],
 'time': [0.6579058170318604,
  0.6233506202697754,
  0.5815865993499756,
  0.5859220027923584,
  0.5579931735992432],
 'means': [99.02773, 0.8834802, 0.8834802, 0.6013516426086426]}

## Scarle pond 23

In [ ]:
hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 1e-3,
    'epochs': 50,
    'lr_reduce_rate': 0.5,
    'patience': 4,
    'early_stopping': 50, # пока уберем раннюю остановку
    'model': 'test'
}
model = UNet(n_filters=hyperparametrs['n_filters'])
model.load_state_dict(torch.load('C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/weight/UNet_spinal-cord/scarlet-pond-23/weights.pth', map_location=torch.device('cpu')))

folder = 'C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/inference/UNet_spinal-cord/scarlet-pond-23/'

metrics = save_predictions_as_imgs(test_loader, model, folder=folder)
save_blended(path_to_folder=folder, number_of_images=5, acc = metrics['accuracy'], h_dice = metrics['DICE'], alpha = 0.75, beta = 0.8)


# print(metrics)
# print("mean accuracy: ", np.mean(metrics[0]))
# print("mean DICE: ", np.mean(metrics[1]))

# save_blended(path_to_folder=folder, number_of_images=5, acc = metrics[0], h_dice = metrics[1], alpha = 0.75)


In [ ]:
metrics

{'accuracy': [tensor(99.1911),
  tensor(98.7632),
  tensor(99.3662),
  tensor(99.1287),
  tensor(99.1074)],
 'soft DICE': [tensor(0.9044),
  tensor(0.8831),
  tensor(0.9136),
  tensor(0.8866),
  tensor(0.8770)],
 'DICE': [tensor(0.9044),
  tensor(0.8831),
  tensor(0.9136),
  tensor(0.8866),
  tensor(0.8770)],
 'time': [2.183528184890747,
  1.9615082740783691,
  2.008281707763672,
  1.9867489337921143,
  2.0976715087890625],
 'means': [99.11131, 0.89293754, 0.89293754, 2.047547721862793]}

In [ ]:
# import json

# print (json.dumps(metrics, indent=2))

# print(metrics["DICE"][0])

12.3 - np.sum(metrics["time"])

# print("mean accuracy: ", np.mean(metrics[0]))
# print("mean DICE: ", np.mean(metrics[1]))

0.9953222751617439

In [ ]:
save_blended(path_to_folder=folder, number_of_images=5, acc = metrics[0], h_dice = metrics[1], alpha = 0.75)


# Comic Wind 42

In [ ]:
hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 1e-3,
    'epochs': 50,
    'lr_reduce_rate': 0.5,
    'patience': 4,
    'early_stopping': 50, # пока уберем раннюю остановку
    'model': 'test'
}
model = UNet(n_filters=hyperparametrs['n_filters'])
model.load_state_dict(torch.load('C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/weight/UNet_spinal-cord/comic-wind-42/weights.pth', map_location=torch.device('cpu')))

folder = 'C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/inference/UNet_spinal-cord/comic-wind-42/'

metrics = save_predictions_as_imgs(test_loader, model, folder=folder)

save_blended(path_to_folder=folder, number_of_images=5, acc = metrics['accuracy'], h_dice = metrics['DICE'], alpha = 0.75, beta = 0.8)


# print(metrics)
# print("mean accuracy: ", np.mean(metrics[0]))
# print("mean DICE: ", np.mean(metrics[1]))

# save_blended(path_to_folder=folder, number_of_images=5, acc = metrics[0], h_dice = metrics[1], alpha = 0.75)


In [ ]:
metrics

{'accuracy': [tensor(99.0889),
  tensor(98.7078),
  tensor(99.5531),
  tensor(99.0107),
  tensor(99.1740)],
 'soft DICE': [tensor(0.8825),
  tensor(0.8652),
  tensor(0.9369),
  tensor(0.8545),
  tensor(0.8874)],
 'DICE': [tensor(0.8825),
  tensor(0.8652),
  tensor(0.9369),
  tensor(0.8545),
  tensor(0.8874)],
 'time': [9.05510401725769,
  10.356470108032227,
  9.091233253479004,
  8.375613689422607,
  8.400790214538574],
 'means': [99.10689, 0.8852884, 0.8852884, 9.05584225654602]}

# Apricot Water 43

In [ ]:
# 2023-02-22_02:40

hyperparametrs = {
    'n_filters': 32,
    'loss_weight': 0.8,
    'lr': 1e-3,
    'epochs': 50,
    'lr_reduce_rate': 0.5,
    'patience': 4,
    'early_stopping': 50, # пока уберем раннюю остановку
    'model': 'test'
}
model = UNet(n_filters=hyperparametrs['n_filters'])
model.load_state_dict(torch.load('C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/weight/UNet_spinal-cord/2023-02-22_02-40/weights.pth', map_location=torch.device('cpu')))

folder = 'C:/Users/gieko/Dropbox/NIITO_Vertebrae/Scripts/inference/UNet_spinal-cord/apricot-water-43/'

metrics = save_predictions_as_imgs(test_loader, model, folder=folder)

# print(metrics)
# print("mean accuracy: ", np.mean(metrics[0]))
# print("mean DICE: ", np.mean(metrics[1]))

save_blended(path_to_folder=folder, number_of_images=5, acc = metrics['accuracy'], h_dice = metrics['DICE'], alpha = 0.75, beta = 0.8)
